In [40]:
import gradio as gr
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import os
from fpdf import FPDF

# Cargar el modelo
model = tf.keras.models.load_model('modelo_clasificacion_basuraUltimateN.h5')

# Detalles de cada clase
class_details = {
    'Aluminium': {'es': 'Aluminio', 'message': 'El aluminio es un material reciclable que se puede reutilizar sin perder calidad.'},
    'Carton': {'es': 'Cartón', 'message': 'El cartón reciclado se usa para hacer nuevos productos como cajas o papel.'},
    'Glass': {'es': 'Vidrio', 'message': 'El vidrio se puede reciclar sin perder calidad, como en botellas y frascos.'},
    'Organic Waste': {'es': 'Residuos Orgánicos', 'message': 'Restos de comida y jardín que se pueden compostar para hacer abono.'},
    'Other Plastics': {'es': 'Otros Plásticos', 'message': 'Plásticos de un solo uso que pueden reciclarse, pero no todos.'},
    'Paper and Cardboard': {'es': 'Papel y Cartón', 'message': 'Papel reciclado para hacer nuevos productos de papel.'},
    'Plastic': {'es': 'Plástico', 'message': 'Plástico reciclado para hacer envases y ropa, pero es importante reducir su uso.'},
    'Textiles': {'es': 'Textiles', 'message': 'Ropa reciclada que se puede convertir en nuevos productos o moda.'},
    'Wood': {'es': 'Madera', 'message': 'Madera reciclada para hacer muebles o energía.'}
}

# Función para hacer predicción
def predict_image(image_path):
    img = image.load_img(image_path)  # Cargar imagen
    img = img.resize((256, 256))  # Redimensionar imagen
    img_array = image.img_to_array(img)  # Convertir imagen a array
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalizar

    prediction = model.predict(img_array)  # Hacer predicción
    predicted_class = np.argmax(prediction)  # Obtener la clase predicha
    confidence = np.max(prediction)  # Obtener la confianza de la predicción

    class_names = os.listdir(os.path.join(os.getcwd(), 'TrashDataset', 'train'))  # Obtener nombres de clases

    predicted_class_name = class_names[predicted_class]
    predicted_class_es = class_details[predicted_class_name]['es']
    message = class_details[predicted_class_name]['message']

    # Crear PDF
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    
    colors = {  
        'Aluminium': (169, 169, 169), 
        'Carton': (255, 140, 0),  
        'Glass': (34, 139, 34),  
        'Organic Waste': (139, 69, 19),  
        'Other Plastics': (255, 69, 0),  
        'Paper and Cardboard': (30, 144, 255),  
        'Plastic': (255, 105, 180),  
        'Textiles': (255, 215, 0),  
        'Wood': (139, 69, 19)  #
    }
    
    color = colors.get(predicted_class_name, (0, 0, 0))  
    pdf.set_text_color(*color)

    pdf.multi_cell(0, 10, f"Clasificación: {predicted_class_es}", align='C')
    pdf.multi_cell(0, 10, f"Confianza: {confidence * 100:.2f}%", align='C')
    pdf.multi_cell(0, 10, f"\n{message}", align='J')

    pdf_output_path = f"{predicted_class_es}_informacion_reciclaje.pdf"
    pdf.output(pdf_output_path)

    return f'🌍 Clase: {predicted_class_es} con confianza de {confidence * 100:.2f}%', pdf_output_path

# Interfaz Gradio
def enhance_interface():
    interface = gr.Interface(fn=predict_image, 
                             inputs=gr.Image(type="filepath", label="📸 Carga una imagen de basura"), 
                             outputs=[gr.Textbox(label="🏆 Resultado de la Clasificación"), 
                                      gr.File(label="📄 Descargar PDF con detalles")],
                             live=True,
                             theme="default",
                             title="♻️ Clasificación de Residuos", 
                             description="🌱 Suba una imagen de basura y el modelo te dirá el tipo de basura y un archivo PDF con más detalles.",
                             submit_btn="Clasificar 🌱", 
                             clear_btn="Limpiar 🧹")

    return interface

# Crear y lanzar la interfaz
interface = enhance_interface()
interface.launch()


* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.
